# Advanced Parametric-Interactive Plots

There are times where `iplot` is not sufficient for our needs. Maybe we need to use specific plot commands not included in the basic backends. Maybe we needs to create more plots connected to the same parameters. Maybe we need to use more complex widgets.

In such cases we will have to use [holoviz's panel](https://panel.holoviz.org/index.html). It is impossible to learn everything `panel` has to offer with a single example, therefore the Reader is encouraged to explore the project website for demos and tutorials.

The following example will focus on SymPy, but the same technique can be used with any other scientific library.

For the purpose of this tutorial, we are going to use Bokeh as the plotting library.

In [ ]:
from sympy import *
init_printing(use_latex=True)
from spb import get_plot_data, plot
from spb.backends.bokeh import BB
import numpy as np
import panel as pn
pn.extension()

Let's suppose we have just computed a symbolic transfer function. We'd like to plot the Bode and Nyquist plots.

In [ ]:
kp, t, z, o = symbols("k_P, tau, zeta, omega")
G = kp / (I**2 * t**2 * o**2 + 2 * z * t * o * I + 1)
G

$\omega$ (the pulsation) is the discretization variable. The parameters are:
* $k_{p}$: proportional gain;
* $\tau$: response time;
* $\zeta$: damping coefficient.

First, we are going to define the widgets that will compose the GUI. For a list of available widgets consult [the following documentation page](https://panel.holoviz.org/user_guide/Widgets.html#types-of-widgets).

In particular, we'd like three float sliders for the aforementioned parameters, and a button to switch from the Bode plot to Nyquist plot.

In [ ]:
kps = pn.widgets.FloatSlider(name="Gain", start=0, end=3, value=1)
ts = pn.widgets.FloatSlider(name="Response Time", start=0, end=3, value=1)
zs = pn.widgets.FloatSlider(name="Damping Coefficient", start=0, end=1, value=0.2)
plot_type = pn.widgets.RadioButtonGroup(
    name="Plot Type", options=["Bode", "Nyquist"], button_type='success')

fig1 = plot(xlabel="[rad / s]", ylabel="Amplitude [dB]", xscale="log", backend=BB, show=False).fig
fig2 = plot(xlabel="[rad / s]", ylabel="Phase [rad]", xscale="log", backend=BB, show=False).fig
fig3 = plot(xlabel="Re(G)", ylabel="Im(G)", backend=BB, show=False).fig

Note that figures are also widgets. Because the Author is not really familiar with Bokeh, he decided to use the `plot` function to quickly initialize the figures (note the keyword argument `show=False`).

Now, we need to create a function that will be called whenever we move the sliders. This function will either add or update the data on the figures.

In [ ]:
@pn.depends(kps, ts, zs, plot_type)
def update(kpval, tval, zval, ptval):
    x, y = get_plot_data(G, (o, -3, 2), params={
        kp: kpval,
        t: tval,
        z: zval,
    }, n=1000, xscale="log")
    
    if ptval == "Bode":
        if len(fig1.renderers) == 0:
                fig1.line(x, 20 * np.log10(abs(y)), line_width=2)
                fig2.line(x, np.angle(y), line_width=2)
        else:
            fig1.renderers[0].data_source.data.update({'y': 20 * np.log10(abs(y))})
            fig2.renderers[0].data_source.data.update({'y': np.angle(y)})
        return pn.Column(
                pn.pane.Bokeh(fig1, height=250),
                pn.pane.Bokeh(fig2, height=250))
    else:
        if len(fig3.renderers) == 0:
            fig3.line(np.real(y), np.imag(y), line_width=2)
            fig3.line(np.real(y), -np.imag(y), line_width=2)
        else:
            fig3.renderers[0].data_source.data.update({'y': np.imag(y)})
            fig3.renderers[1].data_source.data.update({'y': -np.imag(y)})
        return pn.Column(pn.pane.Bokeh(fig3, height=500))

With `@pn.depends(kps, ts, zs, plot_type)` we are explicitely asking for this function to be executed whenever we move the sliders or click the buttons. Note that `update` will receive the values of the specified widgets.

Next, we use `get_plot_data` to extract the numerical data from the symbolic transfer function:
```python
x, y = get_plot_data(G, (o, -3, 2), params={
        kp: kpval,
        t: tval,
        z: zval,
    }, n=1000, xscale="log")
```
Note that we have passed in a dictionary of parameters, similarly to what we would do if we were using `iplot`. Since Bode plots uses a logarithm x-axis, we also specified `xscale="log"` to use a logarithm spacing in the discretization points.

The last thing to note is that the function returns a the objects to be updated. In our case, it will return the figures. The Bode plot is going to use 2 figures, therefore the function returns 2 vertically aligned figures. Nyquist plot will only use one figure.

Finally, we need to create the overall layout. Here, we'll use a left-column containing the sliders and button, and a right-column containing the plots:

In [ ]:
pn.Row(
    pn.Column(kps, ts, zs, plot_type),
    update
)